# Model Training 

## 1. Import Data and required packages

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# Modelling packages
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from xgboost import XGBRegressor
import warnings

In [2]:
df = pd.read_csv('data/StudentsPerformance.csv')

In [3]:
df.head()

,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75


### Preparing X and Y variables

In [4]:
X = df.drop(columns = ['math score'], axis = 1)
Y = df['math score']

In [5]:
X.head()

,gender,race/ethnicity,parental level of education,lunch,test preparation course,reading score,writing score
0,female,group B,bachelor's degree,standard,none,72,74
1,female,group C,some college,standard,completed,90,88
2,female,group B,master's degree,standard,none,95,93
3,male,group A,associate's degree,free/reduced,none,57,44
4,male,group C,some college,standard,none,78,75


In [6]:
Y.head()

0    72
1    69
2    90
3    47
4    76
Name: math score, dtype: int64

### Transform categorical features

In [7]:
num_features = X.select_dtypes(exclude='object').columns
cat_features = X.select_dtypes(include='object').columns

from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

numeric_transformer = StandardScaler()
oh_encoder = OneHotEncoder()

preprocessor = ColumnTransformer(
    [
        ("OneHotEncoder", oh_encoder, cat_features),
        ("StandardScaler", numeric_transformer, num_features),
    ]
)


In [8]:
X = preprocessor.fit_transform(X)

In [9]:
X.shape

(1000, 19)

In [10]:
# separate data into training and testing sets

X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.2, random_state=42)
X_train.shape, X_test.shape, Y_train.shape, Y_test.shape

((800, 19), (200, 19), (800,), (200,))

## Create an evaluation fucntion to give all metrics after model training

In [11]:
def evaluate_regression_models(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mse)
    r2squared = r2_score(true, predicted)
    return {
        'MAE': mae,
        'MSE': mse,
        'RMSE': rmse,
        'R2_Squared': r2squared
    }



In [ ]:
models = {
    "Linear Regression": LinearRegression(),
    "Ridge Regression": Ridge(),
    "Lasso Regression": Lasso(),
    "K-Nearest Neighbors": KNeighborsRegressor(),
    "Decision Tree": DecisionTreeRegressor(),
    "Random Forest": RandomForestRegressor(),
    "Support Vector Regressor": SVR(),
    "AdaBoost Regressor": AdaBoostRegressor(),
    "XGBoost Regressor": XGBRegressor()
}

model_list = []
r2_list = []

for i in range(len(list(models))):
    model = list(models.keys())[i]
    model_name = list(models.values())[i]
    
    # Train the model and predict
    model.fit(X_train, Y_train)
    Y_train_pred = model.predict(X_train)
    Y_test_pred = model.predict(X_test)    
    
    # Evaluate train and test dataset
    train_metrics = evaluate_regression_models(Y_test, Y_test_pred)
    test_metrics = evaluate_regression_models(Y_test, Y_test_pred)
    print(f"Model: {model_name}")
    print(f"Train Metrics: {train_metrics}")
    print(f"Test Metrics: {test_metrics}")
    print("-"*50)
    model_list.append(model_name)
    r2_list.append(test_metrics['R2_Squared'])      

Model: Linear Regression
Train Metrics: {'MAE': 4.214763142474851, 'MSE': 29.095169866715494, 'RMSE': np.float64(5.393993869732843), 'R2_Squared': 0.8804332983749564}
Test Metrics: {'MAE': 4.214763142474851, 'MSE': 29.095169866715494, 'RMSE': np.float64(5.393993869732843), 'R2_Squared': 0.8804332983749564}
--------------------------------------------------
Model: Ridge Regression
Train Metrics: {'MAE': 4.211100688014261, 'MSE': 29.056272192348263, 'RMSE': np.float64(5.390387016935636), 'R2_Squared': 0.880593148502874}
Test Metrics: {'MAE': 4.211100688014261, 'MSE': 29.056272192348263, 'RMSE': np.float64(5.390387016935636), 'R2_Squared': 0.880593148502874}
--------------------------------------------------
Model: Lasso Regression
Train Metrics: {'MAE': 5.157881810347763, 'MSE': 42.5064168384116, 'RMSE': np.float64(6.519694535667419), 'R2_Squared': 0.8253197323627853}
Test Metrics: {'MAE': 5.157881810347763, 'MSE': 42.5064168384116, 'RMSE': np.float64(6.519694535667419), 'R2_Squared': 0.